# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance

In [2]:
s1 = 'ПМ19-5'
s2 = 'ПМ19-4'
edit_distance(s1, s2)

1

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [5]:
word = 'велечайшим'

with open('data/litw-win.txt') as fp:
    words = [line.strip().split()[-1] for line in fp]
words[-5:]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 8: invalid continuation byte

In [8]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('russian')
word = 'попреблагорассмотрительствующемуся'
stemmer.stem(word)

'попреблагорассмотрительств'

In [ ]:
morph = pymorphy2.MorphAnalyzer()
morph.parse(word)[0]

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [12]:
from nltk import sent_tokenize
text = '''Да да да нет'''
sents = sent_tokenize(text)
sents

['Да да да нет']

In [14]:
cv = CountVectorizer()
cv.fit(sents)
cv_sents = cv.transform(sents).toarray()
cv_sents

array([[3, 1]])

In [15]:
cv_sents.shape

(1, 2)

In [17]:
cv.vocabulary_

{'да': 0, 'нет': 1}

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize

data = pd.read_csv('data/preprocessed_descriptions.csv')
data['preprocessed_descriptions'] = data['preprocessed_descriptions'].dropna()
data.head()

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sister in law made these for us at a family...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [2]:
desc_list = data.preprocessed_descriptions.to_list()
tokenized_desc = []
for desc in desc_list:
    try:
        tokenized_desc.append(word_tokenize(desc))
    except:
        continue

In [3]:
flatten_tokenized_desc = []
for desc in tokenized_desc:
    for word in desc:
        flatten_tokenized_desc.append(word)
flatten_tokenized_desc = list(set(flatten_tokenized_desc))

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [5]:
import random
from nltk.metrics.distance import edit_distance

a = random.sample(flatten_tokenized_desc,5)
b = random.sample(flatten_tokenized_desc,5)

for i in range(len(a)):
    print(f'{a[i]},{b[i]}',end = ' ')
    print(edit_distance(a[i],b[i]))

druker,midwesterner 9
deciphering,peninsula 9
davidson,christi 6
liqueur,secret 6
coca,vintners 8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [6]:
import numpy as np

def closest_neighbors(word = 'lol',k = 10,words = flatten_tokenized_desc):
    distances = np.zeros(len(words))
    for i in range(len(words)):
        distances[i] = edit_distance(word,words[i])
    for i in np.argsort(distances)[:k]:
        print(f'Индекс слова:{i}, Расстояние:{distances[i]} {words[i]} {word}')
closest_neighbors('disgusting',10,flatten_tokenized_desc)

Индекс слова:14119, Расстояние:0.0 disgusting disgusting
Индекс слова:4386, Расстояние:2.0 disguising disgusting
Индекс слова:895, Расстояние:2.0 digesting disgusting
Индекс слова:15206, Расстояние:2.0 discussing disgusting
Индекс слова:17179, Расстояние:3.0 disturbing disgusting
Индекс слова:20675, Расстояние:3.0 disgusted disgusting
Индекс слова:8745, Расстояние:3.0 dusting disgusting
Индекс слова:292, Расстояние:3.0 diluting disgusting
Индекс слова:12412, Расстояние:4.0 disguise disgusting
Индекс слова:7301, Расстояние:4.0 dustin disgusting


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [180]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/blarno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [181]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 


flatten_tokenized_desc_set = list(set(flatten_tokenized_desc))
words = pd.DataFrame(flatten_tokenized_desc_set,columns = ['word'])
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
words['stemmed_word'] = words['word'].apply(stemmer.stem)
words['normalized_word'] = words['word'].apply(lemmatizer.lemmatize)
words = words.set_index('word')
words.head()

,stemmed_word,normalized_word
word,,
bowties,bowti,bowtie
braai,braai,braai
chickens,chicken,chicken
certainly,certain,certainly
excellen,excellen,excellen


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [182]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/blarno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [183]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

tokenized_desc_nonstop = []
cnt_stop = 0
cnt_all = 0

for desc in tokenized_desc:
    cur_desc = []
    for word in desc:
        if(word not in stop_words):
            cur_desc.append(word)
        else:
            cnt_stop += 1
        cnt_all += 1
    tokenized_desc_nonstop.append(cur_desc)
print(cnt_stop)
print(cnt_all)
print(cnt_stop/cnt_all)

520134
1103669
0.4712771673391207


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer

sample = data.sample(5).reset_index()
sample_list = sample['preprocessed_descriptions'].to_list()
vectorized = []
vectorizer = TfidfVectorizer()

vectorized = vectorizer.fit_transform(sample_list)
vectorized

<5x67 sparse matrix of type '<class 'numpy.float64'>'
	with 91 stored elements in Compressed Sparse Row format>

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [173]:
from scipy.spatial.distance import cosine

matrix = []
for i in vectorized:
    cur_row = []
    for j in vectorized:
        cur_row.append(cosine(i.toarray(),j.toarray()))
    matrix.append(cur_row)
pd.DataFrame(matrix,columns = sample['name'],index = sample['name'])

name,hot cocoa for a crowd,taco meat seasoning,kahlua hot chocolate,cheesy bacon dip,summer fruit daiquiris alcohol or non alcohol
name,,,,,
hot cocoa for a crowd,0.000000,1.0,0.967428,0.968189,0.955685
taco meat seasoning,1.000000,0.0,1.000000,1.000000,1.000000
kahlua hot chocolate,0.967428,1.0,0.000000,0.967428,1.000000
cheesy bacon dip,0.968189,1.0,0.967428,0.000000,0.955685
summer fruit daiquiris alcohol or non alcohol,0.955685,1.0,1.000000,0.955685,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [174]:
# Тк мы смотрим на косинусное расстояние, а не на косинусное сходство => чем ближе оно к нулю, тем более похожи тексты.
# На диагонали всегда будут нолики, что логично. А вот Конкретно на этой выборке совпали значения между 
# summer fruit daiquiris alcohol or non alcohol/hot cocoa for a crowd(0.955685) и cheesy bacon dip(0.955685)
# => они похожи больше всего